#### 1. R的正交多项式回归

In [1]:
###R语言正交多项式回归
x<-c(37.0, 37.5, 38.0, 38.5, 39.0, 39.5, 40.0, 40.5, 41.0, 41.5, 42.0, 42.5, 43.0)
y<-c(3.40, 3.00, 3.00, 3.27, 2.10, 1.83, 1.53,  1.70, 1.80, 1.90, 2.35, 2.54, 2.90)
lm.pol<-lm(y~1+poly(x,2))
summary(lm.pol)

SyntaxError: invalid syntax. Perhaps you forgot a comma? (1622132883.py, line 4)

In [ ]:
xfit<-seq(37,50,len=5)
yfit<-predict(lm.pol,data.frame(x=xfit))
print(xfit)
print(yfit)

#### 2. 内在非线性回归
R语言做了很多工作，对非线性回归模型做了很大简化

In [ ]:
###内在非线性回归
## 输入数据，构成数据框
cl<-data.frame(
   X=c(rep(2*4:21, c(2, 4, 4, 3, 3, 2, 3, 3, 3, 3, 2, 
       3, 2, 1, 2, 2, 1, 1))),
   Y=c(0.49, 0.49, 0.48, 0.47, 0.48, 0.47, 0.46, 0.46, 
       0.45, 0.43, 0.45, 0.43, 0.43, 0.44, 0.43, 0.43, 
       0.46, 0.45, 0.42, 0.42, 0.43, 0.41, 0.41, 0.40, 
       0.42, 0.40, 0.40, 0.41, 0.40, 0.41, 0.41, 0.40, 
       0.40, 0.40, 0.38, 0.41, 0.40, 0.40, 0.41, 0.38, 
       0.40, 0.40, 0.39, 0.39)
)

## 作非线性拟合，并输出各参数的估计值
nls.sol<-nls(Y~a+(0.49-a)*exp(-b*(X-8)), data=cl,
             start = list( a= 0.1, b = 0.01 ))
summary(nls.sol)

In [ ]:
## 画出拟合曲线和散点图
xfit<-seq(8,44,len=200)
yfit<-predict(nls.sol, data.frame(X=xfit))
plot(cl$X, cl$Y)
lines(xfit,yfit)

## 计算偏导数和相应的Jacobi矩阵
fn<-function(a, b, X){
   f1 <- 1-exp(-b*(X-8))
   f2 <- -(0.49-a)*(X-8)*exp(-b*(X-8))
   cbind(f1,f2)
}
D<-fn(nls.sum$parameters[1,1], nls.sum$parameters[2,1], cl$X)

## 作theta的方差估计
theta.var<-nls.sum$sigma^2*solve(t(D)%*%D); theta.var

#### 3.回归样条

In [ ]:
library(ISLR)
#data("Wage")
attach(Wage)

In [ ]:
library(splines)
agelims=range(age)
age.grid=seq(from=agelims[1],to=agelims[2])
#fit=lm(wage~bs(age,knots=c(25,40,60)),data=Wage)
fit=lm(wage~splines::bs(age,knots=c(25,40,60)),data=Wage)
summary(fit)
pred=predict(fit,newdata=list(age=age.grid),se=T)
plot(age,wage,col="gray")
lines(age.grid,pred$fit,lwd=2)
lines(age.grid,pred$fit+2*pred$se,col='red',lty="dashed")
lines(age.grid,pred$fit-2*pred$se,col='red',lty="dashed")

In [ ]:
library(splines)
agelims=range(age)
age.grid=seq(from=agelims[1],to=agelims[2])
#fit=lm(wage~splines::ns(age,knots=c(25,40,60)),data=Wage)
fit=lm(wage~splines::ns(age,df=5),data=Wage)
summary(fit)
pred=predict(fit,newdata=list(age=age.grid),se=T)
par(pin = c(6,4))
plot(age,wage,col="gray")
lines(age.grid,pred$fit,lwd=3)
lines(age.grid,pred$fit+2*pred$se,col='red',lty="dashed",lwd=3)
lines(age.grid,pred$fit-2*pred$se,col='red',lty="dashed",lwd=3)

- ##### BSplines回归样条和Statsmodels对比

In [ ]:
library(splines)
fit.bs=lm(wage~splines::bs(year,df=4,degree=3)+
          splines::bs(age,df=5,degree=3)+education,data=Wage)
summary(fit.bs)
agelims=range(age)
age.grid=seq(from=agelims[1],to=agelims[2])
plot(age,wage,xlim=agelims,cex=.5,col="darkgrey")
fit=smooth.spline(age,wage,df=5)
fit2=smooth.spline(age,wage,cv=TRUE)
lines(fit,col="red",lwd=2)
lines(fit2,col="blue",lwd=2)